# The Partially Observable Multistage Stochastic Programming Algorithm
Paper: [Partially observable multistage stochastic programming](https://www.sciencedirect.com/science/article/pii/S0167637720300870#!)

Authors: [Oscar Dowson](mailto:oscar.dowson@northwestern.edu), [David P. Morton](mailto:david.morton@northwestern.edu),[Bernardo K. Pagnoncelli](mailto:bernardo.pagnoncelli@northwestern.edu)

-------------------------------------------------------------------------

[Murwan Siddig](mailto:msiddig@clemson.edu)

--------------------------------------------------------------------------
Consider solving the following Multistage Stochastic Programming problem,
\begin{equation}
\min_{x_1 \in \chi_1(x_0,\xi_1)} f_1(x_1, \xi_{1})+ \mathbb{E}_{|\xi_{[1]}}\left[\rule{0cm}{0.4cm} \min_{x_2\in \chi_2(x_1,\xi_2)} f_2(x_2,\xi_{2}) + \mathbb{E}_{|\xi_{[2]}} \left[\rule{0cm}{0.4cm}\cdots + \mathbb{E}_{|\xi_{[T-1]}}\left[\rule{0cm}{0.4cm} \min_{x_T\in \chi_T(x_{T-1},\xi_T)}f_T(x_T,\xi_{T}) \right]\right]\right].
\end{equation}

Here, $\xi_{[t]} := (\xi_1, \dots, \xi_t)$ is a random vector, with a known probability distribution (probability measure) $D_{t}$ supported on a set $\Xi_{t} \subset \mathbb{R}^{n}$, and the set $\Xi_{t}$ is equipped with its Borel sigma-algebra $\mathcal{F}_{t}$.

It is assumed that the candidate distributions $D_t \in \mathcal{D}$ follow a MC with a finite set of Markovian states $\{k\}_{k\in \mathcal{D}}$ and transition probabilities $p_{j}(k) := \mathbb{P}(D_{t+1} = k | D_t = j)$. Now the probability of realization $\xi_{t+1}$ happening, given the Markovian state $D_{t+1} = k$, is given by $q_{k}(\xi_{t+1}) = \mathbb{P}( \xi_{t+1} \in \Xi_{t+1} | D_{t+1} = k)$. 

The following *fully* observable DP formulation can be defined for each state $j \in \mathcal{D}$ in the MC, represented by

\begin{equation}
Q^j_t({x_{t-1}},\xi_{t}):= \left\{
\begin{array}{llll}
\displaystyle \underset{{x_t}}{\min} &  f_t({x_t},\xi_t) + \displaystyle \sum_{k \in \mathcal{D}} p_{j}(k)\cdot \mathcal{Q}^k_{t+1}(x_t)\\
\mbox{s.t.} & \tilde A_t{x_{t-1}} + \tilde B_t{x_t}  = \tilde b_t.
\end{array}
\right.
\end{equation}
where $\mathcal{Q}^k_{t+1}(x_t) := \mathbb{E}_{\xi_{t+1}}[Q^k_{t+1}(x_t,\xi_{t+1})], \; \forall t\neq T$, and $\mathcal{Q}^k_{T+1}(x_T):=0, \; \forall k\in \mathcal{D}$

Since the DM is unable to observe the state of the MC, he/she maintains a probability distribution $\theta_t$ over the set of possible states in the MC. Then, upon observing the realization of the random vector $\xi_t$, the DM updates the \textit{prior} belief vector $\theta_t$ according to the Bayesian update formula, such that the \textit{posterior} probability of being in a state $D_t = j$ is given by 
\begin{equation}
\mathbb{P}(D_{t} = j | \xi_t, \xi_{t-1}, \dots, \xi_1) = \theta_{t}(j) := \frac{\mathbb{P}(D_t = j, \xi_t, \xi_{t-1}, \dots, \xi_1)}{\sum_{k \in \mathcal{D}} \mathbb{P}(D_t = k, \xi_t, \xi_{t-1}, \dots, \xi_1)}
\end{equation}

Consequently, we arrive at the following Partially Observable Multistage Stochastic Programming formulation:

\begin{equation}
Q_t(x_{t-1},\xi_{t}, \theta_{t}):= \left\{
\begin{array}{llll}
\displaystyle \underset{{x_t}}{\min} &  f_t(x_t,\xi_t) + \displaystyle \sum_{j \in \mathcal{D}} {\theta_{t}}(j) \sum_{k \in \mathcal{D}} p_{j}(k)\cdot\mathcal{Q}^k_{t+1}(x_t,\theta_{t})\\
\mbox{s.t.} & \tilde A_t{x_{t-1}} + \tilde B_t{x_t} = \tilde b_t,
\end{array}
\right.
\end{equation}
where the value function $\mathcal{Q}^k_{t+1}(x_t, \theta_{t}):= \mathbb{E}_{\xi_{t+1}}[Q_{t+1}(x_t,\xi_{t+1}, \theta_{t+1})| D_{t+1} = k], \; \forall t\neq T$, and $\mathcal{Q}^k_{T+1}(x_T, \theta_{T}):=0$. 

This can be reformulated as

\begin{equation}
\check{Q}^L_t({x_{t-1}},\xi_{t}, {\theta_{t}}) := \left\{
\begin{array}{llll}
\displaystyle \underset{{x_t}, \mu_{t+1}, \nu_{t+1}}{\min} & f_t({x_t},{\xi_t}) + \mu^\intercal_{t+1}{\theta_t}+ \nu_{t+1} \\
\mbox{s.t.} & \tilde A_t{x_{t-1}} + \tilde B_t{x_t} = \tilde b_t, \\
&  \mu^\intercal_{t+1}{\theta_{t,\ell}} + \nu_{t+1} \geq \alpha_{t+1,\ell} + \beta_{t+1,\ell}^{\intercal} {x_t}, & \forall \ell=1,\dots, L\\
&  \mu^\intercal_{t+1}{\theta_{t,\ell}} + \nu_{t+1}  \geq -M, & \forall \ell=1,\dots, L.
\end{array}
\right.
\end{equation}


In [1]:
#technical lines
using Random;
using CSV;
using DataFrames;
using JuMP;
using Gurobi;
using Distributions;
using LinearAlgebra;
using StatsBase;

In [2]:
#fix the random seed
Random.seed!(2020);
#create gurobi environment
const GRB_ENV = Gurobi.Env();

Academic license - for non-commercial use only - expires 2021-05-08


In [21]:
Φ = CSV.read("MC.csv",DataFrame); #transition probability matrix
#convert the dataframe file to a matrix
Φ = convert(Matrix{Float64}, Φ[:,2:end]);
#The number of states in the Markov Chain Φ
K = size(Φ)[1];

# The long-run average probability matrix (to be used as the initial belief);
Π = Φ^1000;
Π = Π/sum(Π[1,:]);
Π = Π[1,:];

In [126]:
#++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==
HydroP = CSV.read("hydroplant_small.csv",DataFrame);
HydroP = convert(Matrix{Float64}, HydroP[:,2:end]);
H=size(HydroP)[1];
#++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==
ThermoP = CSV.read("thermoplant_small.csv",DataFrame);
ThermoP = convert(Matrix{Float64}, ThermoP[:,2:end]);
F=size(ThermoP)[1];
#++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==
T = 10
R = 5
#++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==
Ξ =  CSV.read("Realizations.csv",DataFrame);
Ξ = convert(Matrix{Float64}, Ξ);

Ω =  CSV.read("distribution.csv",DataFrame);
Ω = convert(Matrix{Float64}, Ω);

#For the stagewise independent case
Ω_SWI = zeros(R);
for r=1:R
    Ω_SWI[r] = sum(Ω[r,:])/size(Ω)[2]
end
########################################################################
# problem formulation related data
########################################################################
#parameter that translates the water flow amount into water level in the reservior
c₀= 2.592;

#demand in stage t
dᵗ = 3000;
#penalty for unsatisfied demand 
cᵖ = 500 #penalty

#maximum allowed amount of turbined flow in hydro plant h in stage t
y̅ᵗ = HydroP[:,1];

#minimum/maximum level of water allowed in hydro plant h
v̲ = HydroP[:,2];
v̅= HydroP[:,3];

#initial amount of water at hydro plant h 
x₀ =  HydroP[:,4].*(v̅-v̲);

v̅ = v̅-v̲;
v̲ = v̲-v̲;

#amount of power generated by releasing one unit of water flow in hydro plant h
rʰ = HydroP[:,5];

#maximum allowed amount of power generated by thermal plant f in stage t
g̅ᵗ = ThermoP[:,1];

#unit cost of thermal power generated from plant f
cᶠ  = ThermoP[:,2];


#The network structure
#Set of all Hydros 
G = [h for h=1:H];

#Set of Hydro plants with reservoir 
Hᴿ = [1, 4, 5, 11, 14, 15];
Hᴿ = filter(x->x<=H,Hᴿ);

#Set of Hydro plants with No reservoir (Run-of river)
Hᴵ = filter(x->x ∉ Hᴿ,G);
Hᴵ = filter(x->x<=H,Hᴵ);

#Graph adjacency list, where U(h) are the set of nodes preceding node h
#set of immediate uper stream hydro plants of h in the network
U = [[],[1],[2],[3],[4],[5],[6],[7],[8],[9],[],[11],[12],[10,13],[14]];
U = U[1:H];

In [127]:
function SAMPLE_2level(B,current_state,t)
    values_ϕ = [k for k=1:K];
    values_ξ = [j for j=1:R];
    j = 0;
    ω = 0;
    if t==1
        j = sample(values_ϕ, Weights(B));
        ω = findmax([sum(Ω[j,:]) for j=1:R])[2];
        ω = 1;
    else
        j = sample(values_ϕ, Weights(Φ[current_state,:]));
        ω = sample(values_ξ, Weights(Ω[:,j]));
    end
    n_pair = (j,ω)
    return n_pair
end

SAMPLE_2level (generic function with 1 method)

In [128]:
function Belief_update(B,j);
    B_temp = B'*Φ;
    B_temp = B_temp/sum(B_temp); #normalization
    for s=1:K
        B[s] = (Ω[j,s]*B_temp[s])/sum(Ω[j,k]*B_temp[k] for k=1:K);
        #if B[s] < 1e-5
        #    B[s]=0;
        #end
    end
    B = B./sum(B);
    B = abs.(B)
    return B
end

Belief_update (generic function with 1 method)

In [129]:
function POSDDP_stage_t_problems(T_s,B);
    x = Array{Any,1}(undef,T_s);
    y = Array{Any,1}(undef,T_s);
    s = Array{Any,1}(undef,T_s);
    g = Array{Any,1}(undef,T_s);
    p = Array{Any,1}(undef,T_s);
    μ = Array{Any,1}(undef,T_s);
    ν = Array{Any,1}(undef,T_s);
    RHS = Array{Any,1}(undef,T_s);
    B_cons = Array{Any,1}(undef,T_s);
    Χ = Array{Any,1}(undef,T_s);
    Θ = Array{Any,1}(undef,T_s);
    
    for t=1:T_s
        subproblem = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));
        # Define the variables.
        @variables(subproblem,
                begin
               v̲[h]  <= xᵗ[h=1:H] <= v̅[h]
                   0 <= yᵗ[h=1:H] <= y̅ᵗ[h]
                   0 <= sᵗ[h=1:H] 
                   0 <= gᵗ[f=1:F] <= g̅ᵗ[f]
                   0 <= pᵗ
                        RHSᵗ[h=1:H]
                        μᵗ[k=1:K]
                        νᵗ                
                end
              );   
        
        #Define the objective.
        if t < T_s
            @objective(subproblem,Min,sum(cᶠ[f]*gᵗ[f] for f=1:F)+cᵖ*pᵗ+sum(B[k]*μᵗ[k] for k=1:K)+νᵗ);
        else
            @objective(subproblem,Min,sum(cᶠ[f]*gᵗ[f] for f=1:F)+cᵖ*pᵗ);
        end
        
        # Define the constraints.        
        B_consᵗ = @constraint(subproblem, sum(B[k]*μᵗ[k] for k=1:K)+νᵗ>=0);
        
        #Flow Balance constraints:
        FB = Array{Any,1}(undef,H);
        #Hydro Plants with reservoir
        for h in Hᴿ
            if length(U[h])>=1
                FB[h]= @constraint(subproblem, xᵗ[h]-c₀*(sum(yᵗ[m]+sᵗ[m] for m in U[h])-(yᵗ[h]+sᵗ[h]))==RHSᵗ[h]);
            else
                FB[h]= @constraint(subproblem, xᵗ[h]-c₀*(-(yᵗ[h]+sᵗ[h]))==RHSᵗ[h]);
            end
        end
        #Hydro Plants with no reservoir
        for h in Hᴵ
            if length(U[h])>=1
                FB[h]= @constraint(subproblem, -(sum(yᵗ[m]+sᵗ[m] for m in U[h])-(yᵗ[h]+sᵗ[h]))==RHSᵗ[h]);
            else
                FB[h]= @constraint(subproblem, -(-(yᵗ[h]+sᵗ[h]))==RHSᵗ[h]);
            end
        end    
        
        #demand constraint
        @constraint(subproblem, sum(rʰ[h]*yᵗ[h] for h=1:H)+sum(gᵗ[f] for f=1:F)+pᵗ>=dᵗ);        
        x[t] = xᵗ;
        y[t] = yᵗ;
        s[t] = sᵗ;
        g[t] = gᵗ;
        p[t] = pᵗ;
        μ[t] = μᵗ;
        ν[t] = νᵗ;
        RHS[t] = RHSᵗ;
        B_cons[t] = B_consᵗ;
        Χ[t] = FB;
        Θ[t] = subproblem; 
    end
    return x, y, s, g, p, μ, ν, RHS, B_cons, Χ, Θ;
end

POSDDP_stage_t_problems (generic function with 1 method)

In [130]:
function POSDDP_forward_pass(xval,ϴ̂,Cᵀx₁,Bᵏ,T_s,x,y,s,g,p,μ,ν,RHS,Χ,B_cons,Θ)
    IS_Path = [];
    Bᵏ_sequance = [];
    push!(Bᵏ_sequance,Bᵏ)
    t = 1;
    i=0;
    n_pair = SAMPLE_2level(Bᵏ,i,t);
    k = n_pair[1];
    lb = 1e-10;
    ub = 1e10;
    
    for t=1:T_s
        if t > 1
            i = IS_Path[t-1]
            n_pair = SAMPLE_2level(Bᵏ,i,t);
            k = n_pair[1];
            j = n_pair[2];            
            for h=1:H
                b̃ᵗ = Ξ[j,h];
                if h in Hᴿ
                    fix(RHS[t][h], xval[t-1,h]+c₀*b̃ᵗ)
                else
                    fix(RHS[t][h], b̃ᵗ)
                end
            end
            
            Bᵘ = Belief_update(deepcopy(Bᵏ_sequance[t-1]),j);
            push!(Bᵏ_sequance,Bᵘ);
            @objective(Θ[t],Min,sum(cᶠ[f]*g[t][f] for f=1:F)+cᵖ*p[t]+sum(Bᵘ[kk]*μ[t][kk] for kk=1:K)+ν[t]);
            for kk=1:K
                set_normalized_coefficient(B_cons[t], μ[t][kk], Bᵘ[kk])
            end
            
        end
        optimize!(Θ[t]);
        status = termination_status(Θ[t]);
        if termination_status(Θ[t]) != MOI.OPTIMAL
            
            println(" in Forward Pass")
            println("Model in stage ", t, " in forward pass is ", status)
            println("Bᵏ_sequance[t] = ", Bᵏ_sequance[t])
            println("Θ[t] = ", Θ[t])
            #exit(0)
        else
            xval[t,:] = value.(x[t]);
          
            if t == 1
                lb = objective_value(Θ[t]);
                Cᵀx₁ = sum(cᶠ[f]*value.(g[t])[f] for f=1:F)+cᵖ*value.(p[t]);
                ϴ̂[t] = sum(Bᵏ[kk]*value.(μ[t])[kk] for kk=1:K)+value.(ν[t])
            else
                ϴ̂[t] = sum(Bᵘ[kk]*value.(μ[t])[kk] for kk=1:K)+value.(ν[t])
            end
        end
        push!(IS_Path,k);
        
    end
    return xval, ϴ̂, Cᵀx₁, Bᵏ_sequance, lb
end

POSDDP_forward_pass (generic function with 1 method)

In [131]:
function POSDDP_backward_pass(xval,ϴ̂,T_s,x,y,s,g,p,μ,ν,RHS,Χ,Θ,B_cons,Bᵏ_sequance)
    Q = zeros(R);
    Q_prob = zeros(R);
    π = zeros(H,R);
    
    for t=T_s:-1:2
        Bᵏ = deepcopy(Bᵏ_sequance[t-1])
        Q_prob = zeros(R);
        for j=1:R
            for i=1:K, k=1:K
                Q_prob[j] += Bᵏ[i]*Φ[i,k]*Ω[j,k];
            end
            #if Q_prob[j] < 1e-5
            #    Q_prob[j] = 0;
            #end
        end
        Q_prob = Q_prob/sum(Q_prob);
        
        for j=1:R
            if Q_prob == 0
                continue 
            else
                
                for h=1:H
                    b̃ᵗ = Ξ[j,h];
                    if h in Hᴿ
                        fix(RHS[t][h], xval[t-1,h]+c₀*b̃ᵗ)
                    else
                        fix(RHS[t][h], b̃ᵗ)
                    end
                end
                Bᵘ = Belief_update(deepcopy(Bᵏ),j);
                @objective(Θ[t],Min,sum(cᶠ[f]*g[t][f] for f=1:F)+cᵖ*p[t]+sum(Bᵘ[kk]*μ[t][kk] for kk=1:K)+ν[t]);
                for kk=1:K
                    set_normalized_coefficient(B_cons[t], μ[t][kk], Bᵘ[kk])
                end
                
                
                optimize!(Θ[t]);
                status = termination_status(Θ[t])
                if termination_status(Θ[t]) != MOI.OPTIMAL
                    println(" in Backward Pass")
                    println("Model in stage ", t, " in Backward pass is ", status)
                    println("Bᵘ = ", Bᵘ)
                    println("Θ[t] = ", Θ[t])
                    #exit(0)
                end    
                Q[j]=objective_value(Θ[t]);
                for h=1:H
                    π[h,j]= dual(Χ[t][h]);
                end
            end
        end
        Q̌ = sum(Q[j]*Q_prob[j] for j=1:R);
        if (Q̌-ϴ̂[t-1])/max(1e-10,abs(ϴ̂[t-1])) > ϵ 
            @constraint(Θ[t-1],
            sum(Bᵏ[kk]*μ[t-1][kk] for kk=1:K)+ν[t-1]-sum(sum(π[h,j]*x[t-1][h] for h in Hᴿ)*Q_prob[j] for j=1:R)
            >=
            Q̌-sum(sum(π[h,j]*xval[t-1,h] for h in Hᴿ)*Q_prob[j] for j=1:R)
            );
            @constraint(Θ[t-1], sum(Bᵏ[kk]*μ[t-1][kk] for kk=1:K)+ν[t-1]>=0);
        end 
        
    end
end

POSDDP_backward_pass (generic function with 1 method)

In [132]:
# define the model 
 x, y, s, g, p, μ, ν, RHS, B_cons, Χ, Θ = POSDDP_stage_t_problems(T,Π);

In [133]:
x_0 = deepcopy(x₀);

j=findmax(deepcopy(Π))[2]


B₀ = zeros(K);
B₀[j] = 1;
B₀ = Π

B = deepcopy(B₀);
Bᵏ = deepcopy(B₀); #The belief at the first stage to be used in case POSDDP
for h=1:H
    b̃ᵗ = Ξ[j,h];
    if h in Hᴿ
        fix(RHS[1][h], x_0[h]+c₀*b̃ᵗ)
    else
        fix(RHS[1][h], b̃ᵗ)
    end
    @objective(Θ[1],Min,sum(cᶠ[f]*g[1][f] for f=1:F)+cᵖ*p[1]+sum(Bᵏ[kk]*μ[1][kk] for kk=1:K)+ν[1]);
    for kk=1:K
        set_normalized_coefficient(B_cons[1], μ[1][kk], Bᵏ[kk])
    end
end

In [134]:
ϵ = 1e-5;
stall =500;
#++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==++==
max_iter = 100000;
nbhrs = 3;
time_limit = 60^2*nbhrs;

In [ ]:
#initialize everything:
LB = [];
Cᵀx₁ = 0;
xval = zeros(T,H);
ϴ̂ = zeros(T);
Relative_Gap = 1e10;
iter = 0;

start=time();
while true
    iter +=1
    xval, ϴ̂, Cᵀx₁, Bᵏ_sequance, lb = POSDDP_forward_pass(xval,ϴ̂,Cᵀx₁,Bᵏ,T,x,y,s,g,p,μ,ν,RHS,Χ,B_cons,Θ)

    push!(LB,lb)
    println("LB = ", lb)

    #termination check:
    if iter > stall
        Relative_Gap = (LB[iter]-LB[iter-stall])/max(1e-10,abs(LB[iter-stall]));
    end
    Elapsed = time() - start;

    if Relative_Gap < ϵ ||  iter > max_iter || Elapsed>time_limit
        if Relative_Gap > ϵ
            if iter > max_iter
                println("terminated because iteration limit was hit")
            else
                println("terminated because time limit was hit")
            end
        end
        break
    else
        #backward_pass: add cuts to improve the cutting plane approxomation of ϴ
        POSDDP_backward_pass(xval,ϴ̂,T,x,y,s,g,p,μ,ν,RHS,Χ,Θ,B_cons,Bᵏ_sequance)

    end
end 

LB = 0.0
LB = 0.0
LB = 0.0
LB = 0.0
LB = 0.0
LB = 0.0
LB = 0.0
LB = 0.0
LB = 0.0
LB = 0.0
LB = 0.0
LB = 0.0
LB = 0.0
LB = 0.0
LB = 1.2447419855475668
LB = 1.2447419855475668
LB = 1.2447419855475668
LB = 1.2447419855475668
LB = 1.2447419855475668
LB = 1.2447419855475668
LB = 5.0014753889622945
LB = 11.200677441763691
LB = 16.741774360470178
LB = 16.741774360470178
LB = 17.19534844666019
LB = 17.92985229951511
LB = 18.461949569862863
LB = 20.1382259609683
LB = 91.96116507975367
LB = 95.409384351157
LB = 100.13130431574669
LB = 100.13130431574669
LB = 106.62127143494541
LB = 120.74506964154358
LB = 121.82545096633504
LB = 129.23020870000255
LB = 199.9099974407079
LB = 199.9099974407079
LB = 248.8558545534986
LB = 367.55575711573385
LB = 367.55575711573385
LB = 367.55575711573385
LB = 387.31031424532887
LB = 390.99946998728103
LB = 418.27812278429576
LB = 420.89492479740386
LB = 855.6322365740884
LB = 857.8373752198017
LB = 888.8557142057557
LB = 909.8044504948625
LB = 1012.115003530178
LB

LB = 6294.811146837539
LB = 6294.811146837539
LB = 6295.882484763675
LB = 6297.823064815349
LB = 6297.823064815349
LB = 6298.700312311825
LB = 6301.352644619046
LB = 6302.445427521705
LB = 6304.334156011371
LB = 6304.4908092332425
LB = 6304.91630133394
LB = 6313.846122848476
LB = 6325.855730202038
LB = 6327.371747703073
LB = 6331.438315241681
LB = 6361.873506464086
LB = 6370.622284475507
LB = 6371.531065963696
LB = 6371.6133200650875
LB = 6376.602140460949
LB = 6383.68898169808
LB = 6396.833874337932
LB = 6401.656011909257
LB = 6415.191607557863
LB = 6420.810440659185
LB = 6427.187808900573
LB = 6436.816769897749
LB = 6442.119689725732
LB = 6443.129503921955
LB = 6445.708212771689
LB = 6457.091974902058
LB = 6457.24676985704
LB = 6479.465836144651
LB = 6481.115128291256
LB = 6482.609901974898
LB = 6487.6925129202755
LB = 6492.225636704758
LB = 6494.43810217323
LB = 6514.726823857234
LB = 6515.055848477761
LB = 6516.2892291544595
LB = 6521.37817467153
LB = 6521.957983749946
LB = 6521.95

LB = 7096.116703116764
LB = 7096.249326345345
LB = 7096.690704109917
LB = 7097.367957610284
LB = 7097.367957610284
LB = 7097.922728452815
LB = 7097.922728452815
LB = 7098.000010399603
LB = 7100.895945259534
LB = 7107.660737142087
LB = 7108.005249054353
LB = 7108.677305776841
LB = 7108.918573219416
LB = 7109.934920580959
LB = 7110.75433763066
LB = 7110.900418279871
LB = 7110.900418279871
LB = 7111.995535844676
LB = 7112.804830468358
LB = 7113.0296600231595
LB = 7113.0296600231595
LB = 7113.0296600231595
LB = 7113.137446099936
LB = 7117.059899672033
LB = 7121.372481246854
LB = 7121.670245600178
LB = 7131.490973297954
LB = 7131.490973297954
LB = 7131.490973297954
LB = 7133.668375022718
LB = 7133.805831824241
LB = 7134.510534607263
LB = 7138.043231496432
LB = 7138.372125700375
LB = 7140.975344186103
LB = 7141.161983989281
LB = 7143.211332773285
LB = 7143.291118274033
LB = 7143.609272693652
LB = 7144.031754893568
LB = 7144.1310947276115
LB = 7145.2627482700855
LB = 7145.381686045673
LB = 71

LB = 7326.151342393585
LB = 7326.151342393585
LB = 7326.151342393585
LB = 7327.333491935287
LB = 7327.333491935287
LB = 7327.412001420067
LB = 7328.489507761295
LB = 7328.489507761295
LB = 7328.565430760399
LB = 7328.73628074133
LB = 7328.73628074133
LB = 7328.871358482593
LB = 7328.96337053715
LB = 7329.059431081707
LB = 7329.176912391741
LB = 7329.176912391741
LB = 7329.176912391741
LB = 7329.379131763341
LB = 7329.509247503801
LB = 7329.509247503801
LB = 7330.001103925593
LB = 7330.001103925593
LB = 7330.415750557144
LB = 7330.415750557144
LB = 7330.415750557144
LB = 7330.513683392654
LB = 7330.588676615143
LB = 7330.695551480809
LB = 7331.06905325917
LB = 7331.248331896801
LB = 7331.435436124044
LB = 7331.769245143441
LB = 7331.9832382085915
LB = 7332.125665939883
LB = 7332.125665939883
LB = 7332.2152521812795
LB = 7332.2152521812795
LB = 7332.301760855948
LB = 7332.756808378921
LB = 7334.009108652512
LB = 7334.009108652512
LB = 7334.780138237458
LB = 7335.455596267524
LB = 7335.45

LB = 7379.47079830571
LB = 7379.887285910365
LB = 7379.887285910365
LB = 7380.173418592841
LB = 7380.173418592841
LB = 7380.561127331363
LB = 7380.777024454365
LB = 7380.955614142378
LB = 7381.441483448566
LB = 7381.691521420209
LB = 7381.691521420209
LB = 7381.792599930108
LB = 7381.885209278852
LB = 7382.223056934109
LB = 7382.448341733394
LB = 7382.448341733394
LB = 7382.73430981753
LB = 7382.880181371424
LB = 7382.880181371424
LB = 7382.880181371424
LB = 7383.042682980519
LB = 7383.206016646585
LB = 7383.206016646585
LB = 7383.206016646585
LB = 7383.206016646585
LB = 7383.288758129148
LB = 7383.288758129148
LB = 7383.8518771668605
LB = 7383.8518771668605
LB = 7383.8518771668605
LB = 7384.070118944823
LB = 7384.070118944823
LB = 7384.185114558677
LB = 7384.185114558677
LB = 7384.185114558677
LB = 7384.276550439394
LB = 7384.276550439394
LB = 7384.276550439394
LB = 7384.276550439394
LB = 7384.352393417041
LB = 7384.352393417041
LB = 7384.816405417263
LB = 7384.816405417263
LB = 7385.

...

LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.406555600065
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.487724521563
LB = 7480.4

LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.142779080423
LB = 7481.1

LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.388082328258
LB = 7481.3

LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.743539192979
LB = 7481.7

LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.872700091222
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.953935064221
LB = 7481.9